# Approach with GPTQ

In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [2]:
# If you are working in Colab, then consider mounting your assignment folder to your drive
from google.colab import drive
drive.mount('/content/drive')

# and change the path below to point to the assignment folder
%cd /content/drive/MyDrive/project-m3-2024-start-unk-stop-pad

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/project-m3-2024-start-unk-stop-pad'
/content


In [3]:
!BUILD_CUDA_EXT=12.2 pip install -q auto-gptq transformers
!pip install huggingface_hub
!pip install datasets
!pip install optimum
!pip install auto-gptq
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google

In [4]:
import random
import auto_gptq
import json
import time
import torch
import accelerate

import os
from huggingface_hub import login
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from drive.MyDrive.project_m3_2024_start_unk_stop_pad.model.utils import read_json
from datasets import Dataset

# Define base model and output directory
model_id = "openai-community/gpt2"
out_dir = model_id + "-GPTQ-4bit"

# Load quantize config, model and tokenizer
quantize_config = BaseQuantizeConfig(
    bits=8,
    group_size=128,
    damp_percent=0.01,
    desc_act=False,
)
model = AutoGPTQForCausalLM.from_pretrained(model_id, quantize_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [6]:
def extract_question_and_answer_pref_dataset(data):
    extracted_pref_data = {
        "prompt": [],
        "chosen": [],
        "rejected": []
    }

    for sample in data:
        question = sample["question_complete"]

        assert isinstance(question, str)

        for pref in sample["preference"]:
            answer_a = pref["A"]
            answer_b = pref["B"]
            overall_score = pref["overall"]

            if overall_score is None:
                continue

            a_pref = "chosen" if "A" in overall_score else "rejected"
            b_pref = "chosen" if "B" in overall_score else "rejected"

            # skip if we do not have a preference
            if a_pref == b_pref:
                continue

            extracted_pref_data["prompt"].append(question)
            extracted_pref_data[a_pref].append(answer_a)
            extracted_pref_data[b_pref].append(answer_b)

    return extracted_pref_data


hf_token = ""  # add your token here
print("logging in")
os.environ["HF_TOKEN"] = hf_token
login(hf_token)

print("Creating dataset from json file")
data = read_json("/content/drive/MyDrive/project_m3_2024_start_unk_stop_pad/data/M1_preference_data_15052024.json")
preference_dataset_dict = extract_question_and_answer_pref_dataset(data)

dataset_no_split = Dataset.from_dict(preference_dataset_dict)
train_test_split = dataset_no_split.train_test_split(test_size=0.2, shuffle=True, seed=0)

test_valid_split = train_test_split['test'].train_test_split(test_size=0.5, shuffle=True, seed=0)

# Combine the splits into a dictionary
dataset = {
    'train': train_test_split['train'],
    'val': test_valid_split['train'],
    'test': test_valid_split['test']
}

dataset['train'].to_json('datasets/dpo_train_dataset.jsonl', orient='records', lines=True)
dataset['val'].to_json('datasets/dpo_val_dataset.jsonl', orient='records', lines=True)
dataset['test'].to_json('datasets/dpo_test_dataset.jsonl', orient='records', lines=True)


logging in
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Creating dataset from json file


Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

6913771

In [8]:
# Load data and tokenize examples
n_samples = 1024
data = dataset['train'][:n_samples * 5]

tokenized_data = tokenizer("\n\n".join(data['chosen']), return_tensors='pt')

# Format tokenized examples
examples_ids = []
for _ in range(n_samples):
    i = random.randint(0, tokenized_data.input_ids.shape[1] - tokenizer.model_max_length - 1)
    j = i + tokenizer.model_max_length
    input_ids = tokenized_data.input_ids[:, i:j]
    attention_mask = torch.ones_like(input_ids)
    examples_ids.append({'input_ids': input_ids, 'attention_mask': attention_mask})

Token indices sequence length is longer than the specified maximum sequence length for this model (1542057 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
# Quantize with GPTQ
model.quantize(
    examples_ids,
    batch_size=1,
    use_triton=True,
)

# Save model and tokenizer
model.save_quantized(out_dir, use_safetensors=True)
tokenizer.save_pretrained(out_dir)

INFO - Start quantizing layer 1/12
INFO:auto_gptq.modeling._base:Start quantizing layer 1/12
INFO - Quantizing attn.c_attn in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_attn in layer 1/12...
INFO - Quantizing attn.c_proj in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_proj in layer 1/12...
INFO - Quantizing mlp.c_fc in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_fc in layer 1/12...
INFO - Quantizing mlp.c_proj in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_proj in layer 1/12...
INFO - Start quantizing layer 2/12
INFO:auto_gptq.modeling._base:Start quantizing layer 2/12
INFO - Quantizing attn.c_attn in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_attn in layer 2/12...
INFO - Quantizing attn.c_proj in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_proj in layer 2/12...
INFO - Quantizing mlp.c_fc in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_fc in layer 2/12...
INFO - Qu

('openai-community/gpt2-GPTQ-4bit/tokenizer_config.json',
 'openai-community/gpt2-GPTQ-4bit/special_tokens_map.json',
 'openai-community/gpt2-GPTQ-4bit/vocab.json',
 'openai-community/gpt2-GPTQ-4bit/merges.txt',
 'openai-community/gpt2-GPTQ-4bit/added_tokens.json',
 'openai-community/gpt2-GPTQ-4bit/tokenizer.json')

In [9]:
#test functionality
# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

result = generator("I have a dream", do_sample=True, max_length=50)[0]['generated_text']
print(result)


The model 'GPT2GPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM

I have a dream that sometimes when I dream it literally makes me want to go to sleep. I would hope most of the time it just makes me want to go to sleep. There are times when I actually do go but it is so weird sometimes


In [29]:
# Access and print the model configuration
config = model.config
print("\nModel Configuration:")
print(config)


Model Configuration:
GPT2Config {
  "_name_or_path": "mNLP-project/gpt2-finetuned-mcqa-sciq2-safety",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "quantization_config": {
    "bits": 8,
    "damp_percent": 0.01,
    "desc_act": false,
    "group_size": 128,
    "is_marlin_format": false,
    "model_file_base_name": null,
    "model_name_or_path": null,
    "quant_method": "gptq",
    "static_groups": false,
    "sym": true,
    "true_sequential": true
  },
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_

the lm_head layer is an linear-layer tied to the embedding on top of the GPT-2 model

In [8]:
#test functionality without generator module
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

inputs = tokenizer("Hello world", return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    outputs = model.generate(**inputs)

print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello world, many thanks for another great question! Hope that was a welcome one! Hope you all have an easier time here! First of all, novelists often ask “how’s it done” and often they ask�


In [28]:
print(tokenizer.decode(model.generate(**tokenizer("auto_gptq is", return_tensors="pt").to(model.device))[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


auto_gptq is not the only way. It also is not the only way. In some areas it could lead to heart attacks or strokes. It is not usually fatal. > The most commonly used is beta-blockers. The beta


In [10]:
# push quantized model to Hugging Face Hub.
from huggingface_hub import login

hf_token = ""  # add your token here
print("logging in")
login(hf_token)

repo_id = f"mNLP-project/baseline-gpt2-quantized"
commit_message = f"AutoGPTQ model for gpt2-dpo: {quantize_config.bits}bits, gr{quantize_config.group_size}, desc_act={quantize_config.desc_act}"
model.push_to_hub(repo_id, commit_message=commit_message, use_auth_token=True)
tokenizer.push_to_hub(repo_id, commit_message=commit_message, use_auth_token=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


INFO - Uploading the following files to mNLP-project/baseline-gpt2-quantized: tokenizer_config.json,config.json,quantize_config.json,tokenizer.json,merges.txt,gptq_model-8bit-128g.safetensors,vocab.json,special_tokens_map.json
INFO:auto_gptq.modeling._base:Uploading the following files to mNLP-project/baseline-gpt2-quantized: tokenizer_config.json,config.json,quantize_config.json,tokenizer.json,merges.txt,gptq_model-8bit-128g.safetensors,vocab.json,special_tokens_map.json


gptq_model-8bit-128g.safetensors:   0%|          | 0.00/243M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/mNLP-project/baseline-gpt2-quantized/commit/13e4c990c9cc21652ef36dda78cd382f3ced0056', commit_message='AutoGPTQ model for gpt2-dpo: 8bits, gr128, desc_act=False', commit_description='', oid='13e4c990c9cc21652ef36dda78cd382f3ced0056', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import accelerate

repo_id = f"mNLP-project/gpt2-safety-GPTQ-8bit"

model_name_or_path = repo_id

# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Hello world"
system_message = "You are a story writing assistant"
prompt_template = f'''### System:
{system_message}

### User:
{prompt}

### Assistant:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512,min_new_tokens=512)
print(tokenizer.decode(output[0]))


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/243M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




*** Generate:
### System:
You are a story writing assistant

### User:
Hello world

### Assistant:

### Contributor:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:

### Programmer:
: Programmer:

### Programmer:

### Programmer:
